In [1]:
from bs4 import BeautifulSoup
import requests
import re
import time
import pandas as pd

agent = 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) \
        Gecko/20100101 Firefox/24.0'
headers = {'User-Agent': agent}

import sqlite3
conn = sqlite3.connect('songs_db.sqlite')

In [117]:
from bs4 import BeautifulSoup
import requests
import time
lyrics_url = "https://www.azlyrics.com/lyrics/ladygaga/pokerface.html"
response = requests.get(lyrics_url, headers=headers)


In [122]:
soup = BeautifulSoup(response.text, 'lxml')
lyrics = soup.find('div', class_=None).text
lyrics = lyrics.replace('\n', ' ').replace('\r', ' ')

x = soup.find('h1')
songTitle = x.text.split(' lyrics')[0]
songTitle = songTitle.replace('\n', ' ').replace('\r', ' ').replace('"', '')
y = soup.find('h2')
artistName = y.text.split(' Lyrics')[0]
artistName = artistName.replace('\n', ' ').replace('\r', ' ').replace('"', '')
print(artistName, songTitle)


Lady Gaga Poker Face


In [181]:
artist_url = "https://www.azlyrics.com/k/knowles.html"
response = requests.get(artist_url, headers=headers)

soup = BeautifulSoup(response.text, 'lxml')
albums_div = soup.find_all('div', class_="listalbum-item")
inputs = soup.find('input', attrs={'type':"hidden", 'name':'artist'})
inputs.attrs['value']

'Beyonce'

In [182]:
songTitle = []
songArtist = []
songUrl = []
songLyrics = []
for link in albums_div:
    song_url = link.find('a')['href'].replace('..','https://www.azlyrics.com')
    response = requests.get(song_url, headers=headers)
    soup = BeautifulSoup(response.text, 'lxml')
    lyrics = soup.find('div', class_=None).text
    
    songLyrics.append(lyrics.replace('\n', ' ').replace('\r', ' '))
    songTitle.append(link.text)
    songArtist.append(inputs.attrs['value'])
    songUrl.append(song_url)
    print(link.text)
    time.sleep(3)
    

Crazy In Love
Naughty Girl
Baby Boy
Hip Hop Star
Be With You
Me, Myself & I
Yes
Signs
Speechless
That's How You Like It
Closer I Get To You
Dangerously In Love 2
Beyonce Interlude
Gift From Virgo
'03 Bonnie & Clyde
Work It Out
Daddy
Fighting Temptations
Fever
Everything I Do
Swing Low, Sweet Chariot
He Still Loves Me
Time To Come Home
Summertime (w/ P. Diddy)
DÃ©jÃ  Vu
Get Me Bodied
Suga Mama
Upgrade U
Ring The Alarm
Kitty Kat
Freakum Dress
Green Light
Irreplaceable


AttributeError: 'NoneType' object has no attribute 'text'

In [175]:
df = pd.DataFrame(
    {
        "SONG_URL": songUrl,
        "SONG_TITLE" : songTitle,
        "SONG_ARTIST" : songArtist,
        "SONG_LYRICS" : songLyrics
    }
)

In [178]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///songs_db.sqlite', echo=False)
df.to_sql('songs', con=engine, if_exists='append', index=False)

In [3]:
# https://github.com/johnwmillr/LyricsGenius
import lyricsgenius
genius = lyricsgenius.Genius("CGUUXgD_-Bz4ISx2joLBXq9d4jDZisGme3f3-Ygb9a2UmEuwol5eWiOnBgPcY-z7")
artist = genius.search_artist("Beyonce", max_songs=3)

Searching for songs by Beyonce...

Changing artist name to 'Beyoncé'
Song 1: "Drunk in Love"
Song 2: "Formation"
Song 3: "Partition"

Reached user-specified song limit (3).
Done. Found 3 songs.


In [5]:
artist.songs

[('Drunk in Love', 'Beyoncé'),
 ('Formation', 'Beyoncé'),
 ('Partition', 'Beyoncé')]

In [9]:
print(artist.songs[0].song_art_image_url)

https://images.genius.com/6b18da0bea43c85d97714d38ec388ccc.648x648x1.jpg


In [239]:
songID = []
songUrl = []
songTitle = []
songArtist = []
songAlbum = []
songYear = []
songImage = []
songLyrics = []

for song in artist.songs:
    songID.append(song._id)
    songUrl.append(song.url)
    songTitle.append(song.title)
    songArtist.append(song.artist)
    songAlbum.append(song.album)
    songYear.append(song.year)
    songImage.append(song.song_art_image_url)
    songLyrics.append(song.lyrics.replace('\n', ' ').replace('\r', ' '))
    
    
#     song = genius.search_song(i[0], i[1])
#     print(song.lyrics.replace('\n', ' ').replace('\r', ' '))

In [240]:
df = pd.DataFrame(
    {
        "SONG_ID" : songID,
        "SONG_URL": songUrl,
        "SONG_TITLE" : songTitle,
        "SONG_ARTIST" : songArtist,
        "SONG_ALBUM" : songAlbum,
        "SONG_YEAR" : songYear,
        "SONG_IMG" : songImage,
        "SONG_LYRICS" : songLyrics
    }
)


In [241]:
df

,SONG_ID,SONG_URL,SONG_TITLE,SONG_ARTIST,SONG_ALBUM,SONG_YEAR,SONG_IMG,SONG_LYRICS
0,299177,https://genius.com/Beyonce-drunk-in-love-lyrics,Drunk in Love,Beyoncé,BEYONCÉ,2013-12-17,https://images.genius.com/6b18da0bea43c85d9771...,"[Verse 1: Beyoncé] I've been drinkin', I've be..."
1,2419257,https://genius.com/Beyonce-formation-lyrics,Formation,Beyoncé,Lemonade,2016-02-06,https://images.genius.com/18193342e63bf90efbde...,[Intro: Messy Mya] What happened at the New Wi...
2,299325,https://genius.com/Beyonce-partition-lyrics,Partition,Beyoncé,BEYONCÉ,2013-12-13,https://images.genius.com/8bf0e48b6e8580d8927a...,"[Part 1: ""Yoncé""] [Intro] Let me hear you say..."
3,299098,https://genius.com/Beyonce-mine-lyrics,Mine,Beyoncé,BEYONCÉ,2013-12-13,https://images.genius.com/69ccd02a122de24f2f2f...,[Verse 1: Beyoncé] I've been watching for the ...
4,2461219,https://genius.com/Beyonce-hold-up-lyrics,Hold Up,Beyoncé,Lemonade,2016-04-23,https://images.genius.com/4c664104206b6a1246d9...,"[Chorus] Hold up, they don't love you like I l..."
...,...,...,...,...,...,...,...,...
378,5414672,https://genius.com/Beyonce-naughty-girl-lil-fl...,Naughty Girl (Lil Flip Remix),Beyoncé,None,None,https://images.genius.com/6850785b48a7d1783c4f...,[Intro: Lil Flip] Ladies and gentlemen! This i...
379,4852408,https://genius.com/Beyonce-haunted-rodg-privat...,Haunted (Rodg Private Mix),Beyoncé,None,2014-01-01,https://images.genius.com/6850785b48a7d1783c4f...,"[x2] It's what you do, it's what you see I kno..."
380,5465766,https://genius.com/Beyonce-ring-the-alarm-span...,Ring the Alarm (Spanglish Version),Beyoncé,None,2008-01-03,https://images.genius.com/95611b4c3407f3b8519b...,"[Refrain] Suena la alarma, seré furiosa No qui..."
381,5120905,https://genius.com/Beyonce-green-light-hba-rem...,Green Light (HBA Remix),Beyoncé,Swaggot Trilltape,2012-01-01,https://images.genius.com/876c5df2c5334828ff4c...,[Intro: Beyoncé] Green light [Verse: Beyoncé]...


In [242]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///songs_db.sqlite', echo=False)
df.to_sql('songs', con=engine, if_exists='append', index=False)

In [11]:
import requests

def request_song_info(query_string):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + 'iPRpGvyPEPwHqexfQo75LsL0i2pPQxhkw-P5WStYbdvmUq-PQyf7ppCnT92Z-ZQc'}
    search_url = base_url + '/search'
    data = {'q': query_string}
    response = requests.get(search_url, data=data, headers=headers)

    return response

In [12]:
def get_song_info(songid):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + 'iPRpGvyPEPwHqexfQo75LsL0i2pPQxhkw-P5WStYbdvmUq-PQyf7ppCnT92Z-ZQc'}
    search_url = base_url + '/songs/' + str(songid)
    response = requests.get(search_url, headers=headers)

    return response

In [73]:
test = request_song_info('Decks Dark by Radiohead')

In [74]:
genius_base_url = "http://genius.com"
response = request_song_info('Decks Dark by Radiohead')
json = response.json()
remote_song_info = None

for hit in json['response']['hits']:
        print(genius_base_url+hit['result']['url'])
        print(hit['result']['full_title'], hit['result']['id'])
#         if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
#             remote_song_info = hit
#             break

http://genius.comhttps://genius.com/Radiohead-decks-dark-lyrics
Decks Dark by Radiohead 2469506
http://genius.comhttps://genius.com/Drake-ovo-sound-radio-episode-44-tracklist-lyrics
OVO Sound Radio Episode 44 Tracklist by Drake 3102958
http://genius.comhttps://genius.com/S-connoisseur-best-tracks-of-2016-annotated
Best Tracks of 2016 by ​s_connoisseur 2929586
http://genius.comhttps://genius.com/Genius-lists-songs-based-on-stories-annotated
Songs Based On Stories by Genius Lists 1357004
http://genius.comhttps://genius.com/Survey-week-2014-popularity-poll-results-annotated
Popularity Poll Results by Survey Week 2014 (Ft. Jason D. Morris) 349997
http://genius.comhttps://genius.com/Genius-best-songs-of-2016-nominations-annotated
Best Songs of 2016 Nominations by Genius 2891157
http://genius.comhttps://genius.com/Radio-hamburg-radio-hamburg-top-825-lyrics
Radio Hamburg TOP 825 by Radio Hamburg 409534
http://genius.comhttps://genius.com/John-green-looking-for-alaska-annotated
Looking For Ala

In [75]:
import requests

def get_spotify_info(songid):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + 'iPRpGvyPEPwHqexfQo75LsL0i2pPQxhkw-P5WStYbdvmUq-PQyf7ppCnT92Z-ZQc'}
    search_url = base_url + '/songs/' + str(songid)
    response = requests.get(search_url, headers=headers)
    song = response.json()
    spotify_uri = [ a['native_uri'] for a in song['response']['song']['media'] if a['provider'] == 'spotify' ]
    spotify_uri = spotify_uri[0] if len(spotify_uri) >= 1 else 'UNKNOWN';
    return spotify_uri


spotify = get_spotify_info(2469506)
spotify

'UNKNOWN'

In [77]:
song_info = get_song_info(2469506)
song_info.json()

{'meta': {'status': 200},
 'response': {'song': {'annotation_count': 12,
   'api_path': '/songs/2469506',
   'apple_music_id': '1111577965',
   'apple_music_player_url': 'https://genius.com/songs/2469506/apple_music_player',
   'description': {'dom': {'tag': 'root',
     'children': [{'tag': 'p',
       'children': ['This song continues the space narrative on ',
        {'tag': 'em', 'children': ['A Moon Shaped Pool']},
        '. Thom uses metaphors of a UFO and extraterrestrial matter in order to portray the questions and darkness of humanity.']},
      '',
      {'tag': 'p',
       'children': ['The title “Decks Dark” would seem to allude to spaceship “decks” – the deck of a ship. In this case these decks being perhaps unmanned – dark, and lifeless.']},
      '',
      {'tag': 'p',
       'children': ['This is the second Radiohead album to feature a song about the arrival of extraterrestrial spacecraft as the third track. The other is ',
        {'tag': 'em', 'children': ['OK Comput

In [72]:
spotify.replace('spotify:track:', '')

'6RX5iL93VZ5fKmyvNXvF1r'

In [69]:

GENIUS_API_KEY = 'iPRpGvyPEPwHqexfQo75LsL0i2pPQxhkw-P5WStYbdvmUq-PQyf7ppCnT92Z-ZQc'
genius_base_url = 'https://api.genius.com'
headers = {'Authorization': 'Bearer ' + GENIUS_API_KEY}

In [58]:
response

[]

In [54]:
response

'spotify:track:6RX5iL93VZ5fKmyvNXvF1r'

In [44]:
a

['spotify:track:6RX5iL93VZ5fKmyvNXvF1r']

In [10]:
response = get_song_info(2441980)
json = response.json()
song_path = json['response']['song']['path']
song_path

'/Beyonce-irreplaceable-lyrics'

In [12]:
page_url = "http://genius.com"
lyrics_url=song_path
response = requests.get(page_url+lyrics_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
lyrics = soup.find('div', class_='lyrics').get_text()
lyrics = lyrics.replace('\n', ' ').replace('\r', ' ')
print(lyrics)

  [Intro] To the left, to the left To the left, to the left (Mmm)  [Verse 1] To the left, to the left Everything you own in the box to the left In the closet, that's my stuff Yes, if I bought it, please don't touch (Don't touch) And keep talking that mess that's fine But could you walk and talk at the same time And, it's my name that's on that jag So remove your bags, let me call you a cab  [Pre-Chorus] Standing in the front yard Tellin' me, how I'm such a fool Talkin' 'bout, how I'll never ever find a man like you You got me twisted  [Chorus] You must not know about me, you must not know about me I could have another you in a minute Matter of fact, he'll be here in a minute, baby You must not know about me, you must not know about me I can have another you by tomorrow So don't you ever for a second get to thinking You're irreplaceable  [Verse 2] So go ahead and get gone Call up that chick and see if she's home Oops, I bet you thought, that I didn't know What did you think I was puttin

In [14]:
import re
pattern = r'\[.*?\]'
s = """Issachar is a rawboned[a] donkey lying down among the sheep pens.[b]"""
re.sub(pattern, '', lyrics)

"   To the left, to the left To the left, to the left (Mmm)   To the left, to the left Everything you own in the box to the left In the closet, that's my stuff Yes, if I bought it, please don't touch (Don't touch) And keep talking that mess that's fine But could you walk and talk at the same time And, it's my name that's on that jag So remove your bags, let me call you a cab   Standing in the front yard Tellin' me, how I'm such a fool Talkin' 'bout, how I'll never ever find a man like you You got me twisted   You must not know about me, you must not know about me I could have another you in a minute Matter of fact, he'll be here in a minute, baby You must not know about me, you must not know about me I can have another you by tomorrow So don't you ever for a second get to thinking You're irreplaceable   So go ahead and get gone Call up that chick and see if she's home Oops, I bet you thought, that I didn't know What did you think I was putting you out for? 'Cause you was untrue Rollin'

In [53]:
base_url = 'https://api.genius.com'
headers = {'Authorization': 'Bearer ' + 'iPRpGvyPEPwHqexfQo75LsL0i2pPQxhkw-P5WStYbdvmUq-PQyf7ppCnT92Z-ZQc'}
search_url = base_url + '/songs/2441980'
response = requests.get(search_url, headers=headers)

In [81]:
class Song:
    def __init__(self, id, title, artist, url, album, year, image, lyrics ):
        self.id = id
        self.title = title
        self.artist = artist
        self.url = url
        self.album = album
        self.year = year
        self.image = image
        self.lyrics = lyrics

In [85]:
song['response']['song'].keys()

dict_keys(['annotation_count', 'api_path', 'apple_music_id', 'apple_music_player_url', 'description', 'embed_content', 'featured_video', 'full_title', 'header_image_thumbnail_url', 'header_image_url', 'id', 'lyrics_owner_id', 'lyrics_placeholder_reason', 'lyrics_state', 'path', 'pyongs_count', 'recording_location', 'release_date', 'release_date_for_display', 'song_art_image_thumbnail_url', 'song_art_image_url', 'stats', 'title', 'title_with_featured', 'url', 'current_user_metadata', 'album', 'custom_performances', 'description_annotation', 'featured_artists', 'lyrics_marked_complete_by', 'media', 'primary_artist', 'producer_artists', 'song_relationships', 'verified_annotations_by', 'verified_contributors', 'verified_lyrics_by', 'writer_artists'])

In [92]:
song = response.json()
id = song['response']['song']['id']
title = song['response']['song']['title']
artist = song['response']['song']['primary_artist']['name']
artist_img = song['response']['song']['primary_artist']['image_url']
url = song['response']['song']['url']
album = song['response']['song']['album']['name']
year = song['response']['song']['release_date']
song_image = song['response']['song']['song_art_image_thumbnail_url']
lyrics = song['response']['song']['title']